In [222]:
from bs4 import BeautifulSoup
import urllib.request
import json
from urllib import parse
import requests
import nltk

In [223]:
def get_diseases_list(query_list):
    query = ""
    for single_query in query_list:
        query += "/" + single_query

    URL = 'https://www.healthline.com/symptom'
    list_of_diseases = []

    # -------request for first 20 items

    with urllib.request.urlopen(URL + query) as top_url:
        top_page = BeautifulSoup(top_url, 'lxml').find_all('h2', class_="h1 css-1yjq9vb")
        for item in top_page:
            list_of_diseases.append(item.getText())
        
        content = requests.get(URL + query).content.decode().split('relatedSymptoms') # get related symptoms
        relatedSymptoms = content[1].split("sidebarContent")[0]
    # ------request for more items if possible

    headers = { "authority":"www.healthline.com",
                "origin":"https://www.healthline.com",
                "referer":"https://www.healthline.com/symptom/dizziness",
                "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36"
              } # refer to request header in chrome
    has_next = True
    start = 20
    START_ITEM = 20

    while has_next:
        table = {'rows':10,'start':start,'symptoms':query_list}
        text = json.dumps(table)
        quest_url = "https://www.healthline.com/api/symptom-checker/conditions"
        res = requests.post(quest_url, data= text) # simulate a POST request here 
        dic = json.loads(res.content.decode(),encoding = 'utf-8')
        diseases = dic["items"]
        count = dic["conditionCount"]
        if count <= START_ITEM:
            break
        for item in diseases:
            list_of_diseases.append(item["title"][0])
        if start + 10 >= count:
            has_next = False
        start += 10
    
    return list_of_diseases,eval(relatedSymptoms[2:-2])

In [224]:
#Extract the name of disease by nltk pos tag
def extract_name(list_of_diseases):
    disease_pos_tags = ["NNP","JJ","(",")"] # Pos_tag for diseases
    sentence_symbols = ["What","Everything","Is"]
    diseases_list_new = []

    for item in list_of_diseases:
        if "What" in item or "Everything" in item or "Is" in item: #add the rule for detecting sentence
            disease = ""
            tokens = WordPunctTokenizer().tokenize(item)
            tokens = nltk.pos_tag(tokens)
            for word in tokens:
                if word[1] in disease_pos_tags:    # disease format: JJ+NNP, NNP
                    if word[1] == "JJ" and disease != "":
                        continue
                    disease +=word[0] + " "
            diseases_list_new.append(disease)
            continue
        diseases_list_new.append(item)

    return diseases_list_new

In [225]:
# Test the code
#query_list = ["dizziness","fainting"]
query_list = ["dizziness"]

In [226]:
# Test the code 

diseases_list,relatedSymptoms = get_diseases_list(query_list)
print("Number of diseases:",len(diseases_list))
print()
print("Related Symptoms:\n")
for item in relatedSymptoms:
    print(item['cfn'])
print()
print("List of Diseases:\n")

for item in diseases_list:
    print(item)

Number of diseases: 84

Related Symptoms:

Shortness of Breath
Fast Heart Rate
Vomiting
Abnormal Heart Rhythms
Fainting
Heart Rate Altered
Fatigue
Fluttering Heart
Anxiety
General Weakness
Headache
Lightheadedness
Nausea
Palpitations
Pain
Loss of Consciousness
Excessive sweating
Fever
Tired
Low Blood Pressure

List of Diseases:

Low Blood Sugar (Hypoglycemia)
Labyrinthitis
What Is Hypotension?
Meniere's Disease
Everything You Need to Know About High Blood Pressure (Hypertension)
What Causes Abnormal Heart Rhythms?
Everything You Need to Know About Stroke
Heart Attack
Bleeding
What Is Hyperventilation?
Dehydration
Panic Disorder
Motion Sickness
Heat Emergencies
Beriberi
Hypovolemic Shock
Middle Ear Infection (Otitis Media)
Hypothermia
Vertebrobasilar Circulatory Disorders
Cold and Flu
Benign Positional Vertigo (BPV)
Ventricular Tachycardia
Whiplash
Caffeine Overdose: How Much Is Too Much?
Paroxysmal Supraventricular Tachycardia (PSVT)
Atrial Flutter
Epidural Hematoma
What Do You Want to

In [227]:
# Only extract the name of disease 

diseases_list_names = extract_name(diseases_list)

print("Number of diseases:",len(diseases_list_names))
print()
print("List of Diseases:\n")
for item in diseases_list_names:
    print(item)

Number of diseases: 84

List of Diseases:

Low Blood Sugar (Hypoglycemia)
Labyrinthitis
Hypotension 
Meniere's Disease
High Blood Pressure ( Hypertension ) 
Abnormal Heart Rhythms 
Stroke 
Heart Attack
Bleeding
Hyperventilation 
Dehydration
Panic Disorder
Motion Sickness
Heat Emergencies
Beriberi
Hypovolemic Shock
Middle Ear Infection (Otitis Media)
Hypothermia
Vertebrobasilar Circulatory Disorders
Cold and Flu
Benign Positional Vertigo (BPV)
Ventricular Tachycardia
Whiplash
Caffeine Overdose 
Paroxysmal Supraventricular Tachycardia (PSVT)
Atrial Flutter
Epidural Hematoma
Pregnancy 
Atrial Fibrillation and Flutter
Transient Ischemic Attack ( Ministroke ) 
Atherosclerosis
Ectopic Pregnancy
Pulmonary Embolism
Cardiac Tamponade
Abdominal Aortic Aneurysm
Viral Gastroenteritis (Stomach Flu)
Common Cold Overview
Peptic Ulcer
Toxic Megacolon
Anorexia Nervosa
Multifocal Atrial Tachycardia and Your Heart
Hyperemesis Gravidarum
Chemical Burns
Fibromyalgia 
Marine Animal Bites or Stings
Focal Ons